In [ ]:
# Summary of Findings:
# Close to 1 sharp ratio for the entire portfolio
# not too bad but not good enough to use as there are 200 day periods of drawdowns...

In [6]:
import contract_utilities.contract_meta_info as cmi
tickerhead_list = list(set(cmi.futures_butterfly_strategy_tickerhead_list + cmi.cme_futures_tickerhead_list))
tickerhead_list

['GC',
 'BO',
 'KC',
 'AD',
 'SI',
 'RB',
 'KW',
 'CD',
 'B',
 'C',
 'TY',
 'SM',
 'NG',
 'OJ',
 'CL',
 'LN',
 'W',
 'CT',
 'FC',
 'HO',
 'ED',
 'EC',
 'SB',
 'ES',
 'NQ',
 'CC',
 'TU',
 'BP',
 'US',
 'JY',
 'FV',
 'LC',
 'S']

In [1]:
import fundamental_data.cot_data as cd
cd.presave_cot_data()

In [7]:
import backtesting.backtest_swing as bs
results_dictionary = bs.get_results_4tickerhead(ticker_head='NQ')
daily_frame = results_dictionary['daily_frame']
trades_frame = results_dictionary['trades_frame']

# Sharp ratios
# C 0.27 vs -0.19
# W 0.24 vs -0.27
# S 0.37 vs 0.05

# CL 0.21 vs 0.00
# NG -0.02 vs -0.38

# ES 0.34 vs 0.32



In [5]:
import numpy as np
print(16*daily_frame['pnl'].mean()/daily_frame['pnl'].std())
print(16*daily_frame['buy_hold'].mean()/daily_frame['buy_hold'].std())

0.3403782691626233
0.32544681538699194


In [6]:

import matplotlib.pyplot as plt

plt.plot(range(len(daily_frame.index)),daily_frame['pnl'].cumsum(),
         range(len(daily_frame.index)),daily_frame['buy_hold'].cumsum())
plt.grid()         
         
plt.show()

In [59]:
print(trades_frame['pnl'].sum())
print(np.nansum(pnl_path))

2499.25
2485.5


In [40]:
trades_frame['pnl'].mean()

0.04520938023450585

In [1]:
import contract_utilities.expiration as exp
date_list = exp.get_bus_day_list(date_from=20140101, date_to=20170317)

In [8]:
import opportunity_constructs.arma_strategy as arma

backtest_output_list = []

for i in range(len(date_list)):
    print(date_list[i])
    arma_output = arma.get_arma_sheet_4date(date_to=date_list[i])
    arma_sheet = arma_output['arma_sheet']
    arma_sheet['report_date'] = date_list[i]
    backtest_output_list.append(arma_sheet)

20140102
20140103
20140106
20140107
20140108
20140109
20140110
20140113
20140114
20140115
20140116
20140117
20140121
20140122
20140123
20140124
20140127
20140128
20140129
20140130
20140131
20140203
20140204
20140205
20140206
20140207
20140210
20140211
20140212
20140213
20140214
20140218
20140219
20140220
20140221
20140224
20140225
20140226
20140227
20140228
20140303
20140304
20140305
20140306
20140307
20140310
20140311
20140312
20140313
20140314
20140317
20140318
20140319
20140320
20140321
20140324
20140325
20140326
20140327
20140328
20140331
20140401
20140402
20140403
20140404
20140407
20140408
20140409
20140410
20140411
20140414
20140415
20140416
20140417
20140421
20140422
20140423
20140424
20140425
20140428
20140429
20140430
20140501
20140502
20140505
20140506
20140507
20140508
20140509
20140512
20140513
20140514
20140515
20140516
20140519
20140520
20140521
20140522
20140523
20140527
20140528
20140529
20140530
20140602
20140603
20140604
20140605
20140606
20140609
20140610
20140611
2

In [9]:
import pandas as pd
backtest_frame = pd.concat(backtest_output_list)
backtest_frame['normalized_target_'] = -backtest_frame['normalized_target']

In [4]:
import backtesting.backtest_stats as bs
bs.get_indicator_rr_table(trade_data=backtest_frame,indicator_name='normalized_forecast',
                          strategy_class='arma',
                          long_pnl_field='normalized_target',
                                            short_pnl_field='normalized_target_')



,indicator_ulimit,mean_pnl,reward_risk
0,NaN,0.036317,0.039093
1,0.181311,-0.010899,-0.014341
2,0.098933,-0.012916,-0.017980
3,0.045231,0.000819,0.001122
4,0.004766,0.005702,0.008055
5,-0.035347,0.021493,0.031745
6,-0.073091,-0.017615,-0.023847
7,-0.123290,0.054682,0.077177
8,-0.210674,0.108223,0.119226


In [6]:
short_forecast = backtest_frame[backtest_frame['normalized_forecast']<-0.2]
import matplotlib.pyplot as plt

plt.plot(short_forecast['normalized_target_'].cumsum())
plt.grid()         
         
plt.show()

In [24]:
# Portfolio Time Series Properties

date_list = backtest_frame['report_date'].unique()
import numpy as np
short_pnl_list = []
long_pnl_list = []

selected_frame = backtest_frame[backtest_frame['ticker_class']=='Energy']


for i in range(len(date_list)):
    
    daily_data = selected_frame[selected_frame['report_date']==date_list[i]]
    short_data = daily_data[daily_data['normalized_forecast']<-0.2]
    long_data = daily_data[daily_data['normalized_forecast']>0.2]
    
    if short_data.empty:
        short_pnl = 0
    else:
        short_pnl  = short_data['normalized_target_'].sum()
        
    if long_data.empty:
        long_pnl = 0
    else:
        long_pnl  = long_data['normalized_target'].sum()
    
    short_pnl_list.append(short_pnl)
    long_pnl_list.append(long_pnl)
        

        
porfolio_frame = pd.DataFrame.from_items([('report_date', date_list), 
                                          ('short_pnl', short_pnl_list),
                                          ('long_pnl', long_pnl_list)])  


porfolio_frame['total_pnl'] = porfolio_frame['short_pnl']+porfolio_frame['long_pnl']

In [25]:
plt.plot(range(len(porfolio_frame.index)),porfolio_frame['long_pnl'].cumsum(),
         range(len(porfolio_frame.index)),porfolio_frame['short_pnl'].cumsum(),
         range(len(porfolio_frame.index)),porfolio_frame['total_pnl'].cumsum())
plt.grid()         
         
plt.show()

In [26]:
print('Total Sharp:' + str(16*porfolio_frame['total_pnl'].mean()/porfolio_frame['total_pnl'].std()))
print('Long Sharp:' + str(16*porfolio_frame['long_pnl'].mean()/porfolio_frame['long_pnl'].std()))
print('Short Sharp:' + str(16*porfolio_frame['short_pnl'].mean()/porfolio_frame['short_pnl'].std()))

Total Sharp:1.0520938854558173
Long Sharp:0.1375058753965101
Short Sharp:1.108888978857929


In [21]:
# Distribution of Pnl Across Tickerclasses

ticker_class_list = backtest_frame['ticker_class'].unique()
import numpy as np
short_pnl_list = []
long_pnl_list = []

for i in range(len(ticker_class_list)):
    
    daily_data = backtest_frame[backtest_frame['ticker_class']==ticker_class_list[i]]
    short_data = daily_data[daily_data['normalized_forecast']<-0.2]
    long_data = daily_data[daily_data['normalized_forecast']>0.2]
    
    if short_data.empty:
        short_pnl = 0
    else:
        short_pnl  = short_data['normalized_target_'].sum()
        
    if long_data.empty:
        long_pnl = 0
    else:
        long_pnl  = long_data['normalized_target'].sum()
    
    short_pnl_list.append(short_pnl)
    long_pnl_list.append(long_pnl)

porfolio_frame = pd.DataFrame.from_items([('ticker_class', ticker_class_list), 
                                          ('short_pnl', short_pnl_list),
                                          ('long_pnl', long_pnl_list)])  


porfolio_frame['total_pnl'] = porfolio_frame['short_pnl']+porfolio_frame['long_pnl']

porfolio_frame

,ticker_class,short_pnl,long_pnl,total_pnl
0,FX,21.170135,-12.784167,8.385968
1,Energy,145.047611,9.748934,154.796545
2,Ag,1.219883,3.061373,4.281255
3,Soft,1.973559,1.430267,3.403826
4,STIR,0.261572,31.478936,31.740508
5,Index,-4.405431,-10.525375,-14.930806
6,Livestock,9.653632,38.470294,48.123926
7,Treasury,-0.221165,2.817877,2.596712
8,Metal,-1.949422,10.433707,8.484285


In [20]:
backtest_frame

,ticker,ticker_head,ticker_class,forecast,normalized_forecast,param1,param2,normalized_target,report_date,normalized_target_
0,ADH2014,AD,FX,NaN,NaN,0,0,1.284311,20140102,-1.284311
1,BJ2014,B,Energy,NaN,NaN,0,0,-0.889064,20140102,0.889064
2,BOH2014,BO,Ag,NaN,NaN,0,0,-0.465078,20140102,0.465078
3,BPH2014,BP,FX,0.113313,0.251087,2,2,-0.013500,20140102,0.013500
4,CH2014,C,Ag,0.070337,0.041184,2,2,0.417738,20140102,-0.417738
5,CCH2014,CC,Soft,-0.833521,-0.508009,1,1,1.456633,20140102,-1.456633
6,CDH2014,CD,FX,NaN,NaN,0,0,0.000000,20140102,-0.000000
7,CLJ2014,CL,Energy,0.233891,0.170743,1,0,-1.138375,20140102,1.138375
8,CTH2014,CT,Soft,-0.115249,-0.080374,2,0,-0.912816,20140102,0.912816
9,ECH2014,EC,FX,0.074886,0.147814,1,0,-0.780751,20140102,0.780751
